In [40]:
import numpy as np
#from multivar_horner import HornerMultivarPolynomial
import sympy as sp

# Population polynomial - polynomial.f90 : newton

The subroutine calculates the solution for n d-dimensional polynomial equations using the Newton-Raphson method in combination with the Horner method for multivariate polynomials. 
The Horner method is used for the calculation of the value of the polynomial and its derivatives at a given point.

#### $\textbf{Newton-Raphson method using derivatives}$
The Newton-Raphson method is an iterative method for finding the roots of a function. The method requires both the function $f(x)$ and the derivative $f'(x)$ at arbitrary points $x$. The formula consists geometrically of extending the tangent line at a current point $x_i$ until it intersects the x-axis. The intersection point is the next guess $x_{i+1}$. The formula for the one dimensional case is given by: </p>
$$x_{i+1} = x_i - \frac{f(x_i)}{f'(x_i)}$$
This can be expressed in terms of a multiplication of the inverse of the Jacobian matrix with the function vector. The Jacobian matrix is the matrix of all first-order partial derivatives of the function. The formula for the multivariate case is given by: </p>
$$x_{i+1} = x_i - J_f^{-1}(x_i) f(x_i)$$
where $J$ is the Jacobian matrix and $f(x_i)$ is the function vector. The Jacobian matrix is given by: </p>
$$J_f = \begin{bmatrix} \frac{\partial f_1}{\partial x_1} & \frac{\partial f_1}{\partial x_2} & \cdots & \frac{\partial f_1}{\partial x_n} \\ \frac{\partial f_2}{\partial x_1} & \frac{\partial f_2}{\partial x_2} & \cdots & \frac{\partial f_2}{\partial x_n} \\ \vdots & \vdots & \ddots & \vdots \\ \frac{\partial f_n}{\partial x_1} & \frac{\partial f_n}{\partial x_2} & \cdots & \frac{\partial f_n}{\partial x_n} \end{bmatrix}$$

The subroutine calculates the inverse of the Jacobian matrix directly for the dimensions one, two and three. For higher dimensions, the subroutine uses the LU decomposition to calculate the inverse of the Jacobian matrix. The LU decomposition is a method for solving systems of linear equations. The method decomposes the matrix into a lower triangular matrix and an upper triangular matrix. The inverse of the Jacobian matrix is then calculated by solving the linear system for each column of the identity matrix. (Replace by LAPACK?) </p> 

#### $\textbf{Horner method for multivariate polynomials}$
The value of the function and its derivatives at a given point are calculated using the Horner method. The Horner method is a method for evaluating polynomials. The method is based on the factorization of the polynomial. In the one-dimensional case, the polynomial is written as: </p>
$$p(x) = a_0 + a_1 x + a_2 x^2 + \cdots + a_n x^n = a_0 + x(a_1 + x(a_2 + \cdots + x(a_{n-1} + a_n x)))$$
The derivative of the polynomial is calculated in a similar way using the chain rule. The multivariate polynomial is written as: </p>
$$p(x_1, x_2, \cdots, x_n) = a_0 + a_1 x_1 + a_2 x_1^2 + \cdots + a_n x_1^{n_1} + a_{n+1} x_2 + a_{n+2} x_1 x_2 + \cdots + a_{n+m} x_1^{n_1} x_2^{n_2} \cdots x_m^{n_m}$$
This is achieved by recursively evaluating the polynomial for each variable.

#### Helper for finding the order of the polynomial coefficients in the array

In [41]:
# Array telling which coefficient belongs to which index
def coeff_order(nr_monomers, degree, cluster_comp):
    # nr_monomers      : input         - Number of monomers in the system
    # degree           : input         - Array containing the highest degree of each monomer
    

    # n_comp is the number of possible combinations of the monomers in the system
    n_comp = 1
    for i in range(1, len(degree)+1):
        n_comp = n_comp * degree[i-1]

    # coeff_order is the array containing the degree of each monomer for all the possible combinations
    # and all equations. It has the size of [n_comp*nr_monomers, nr_monomers]
    coeff_order = np.zeros((n_comp*nr_monomers, nr_monomers))

    # Loop over all clusters and set the coefficients
    for cluster in range(len(cluster_comp)):
        index = cluster_comp[cluster][0]

        for j in range(1, nr_monomers):
            index += cluster_comp[cluster][j] * np.product(degree[:j])

        for k in range(0, nr_monomers):
            coeff_order[index + k*n_comp] += cluster_comp[cluster]
    
    return coeff_order

In [47]:
# Two monomers, each with a degree of 4
monomers = 2
degree = (4,4)
cluster_comp = [[1,0], [2,0], [3,0], [0,1], [1,1], [2,1], [3,1], [0,2], [1,2], [2,2], [3,2], [0,3], [1,3], [2,3], [3,3]]
coeffs_all = coeff_order(monomers, degree, cluster_comp)

print("Length of coefficient array: ",len(coeffs_all))
print("coefficient array: ", "\n", coeffs_all[16:32])

# Two monomers, each with a degree of 3
monomers = 2
degree = (3,3)
cluster_comp = [[1,0], [2,0], [0,1], [1,1], [2,1], [0,2], [1,2], [2,2]]
coeffs_all = coeff_order(monomers, degree, cluster_comp)

print("Length of coefficient array: ",len(coeffs_all))
print("coefficient array: ", "\n", coeffs_all[0:9])

# Two monomers, each with a degree of 2
monomers = 2
degree = (2,2)
cluster_comp = [[1,0], [0,1], [1,1]]
coeffs_all = coeff_order(monomers, degree, cluster_comp)

print("Length of coefficient array: ",  len(coeffs_all))
print("coefficient array: ", "\n", coeffs_all[0:4])

# One monomer with a degree of 5
monomers = 1
degree = []
degree = [5]
cluster_comp = [[1], [2], [3], [4]]
coeffs_all = coeff_order(monomers, degree, cluster_comp)

#print("Length of coefficient array: " , len(coeffs_all))
#print("coefficient array: ", "\n", coeffs_all[0:5])


Length of coefficient array:  32
coefficient array:  
 [[0. 0.]
 [1. 0.]
 [2. 0.]
 [3. 0.]
 [0. 1.]
 [1. 1.]
 [2. 1.]
 [3. 1.]
 [0. 2.]
 [1. 2.]
 [2. 2.]
 [3. 2.]
 [0. 3.]
 [1. 3.]
 [2. 3.]
 [3. 3.]]
Length of coefficient array:  18
coefficient array:  
 [[0. 0.]
 [1. 0.]
 [2. 0.]
 [0. 1.]
 [1. 1.]
 [2. 1.]
 [0. 2.]
 [1. 2.]
 [2. 2.]]
Length of coefficient array:  8
coefficient array:  
 [[0. 0.]
 [1. 0.]
 [0. 1.]
 [1. 1.]]


### Solve polynomials with sympy

Example polynomials: </p>
$ f(x,y) = -1 + x + y $ </br>
$ g(x,y) = -3 + 2x + 4y $ </p> 

Solution: $[x, y] = [0.5, 0.5]$ </p>

In [43]:
x,y = sp.symbols('x, y')
eqs = [x + y -1, -3 + 2*x + 4*y]
sol = sp.solve(eqs, (x, y))
print(sol)

{x: 1/2, y: 1/2}


Example polynomials: </p>
$ f(x,y) = -1 + x^2 + y^2 $ </br>
$ g(x,y) = 2x - y $ </p> 

Solution: $[x, y] = [ \frac{1}{\sqrt{5}},  \frac{2}{\sqrt{5}}$] </p>
$~~~~~~~~~~~~~~~~~~~~~~~~~~~\Bigg( [- \frac{1}{\sqrt{5}}, - \frac{2}{\sqrt{5}}] \Bigg)$ </p>

In [44]:
x,y = sp.symbols('x, y')
eqs = [x**2 + y**2 -1, 2*x - y]
sol = sp.solve(eqs, (x, y))

#Show only the with x and y between 0 and 1
sol = [s for s in sol if 0 <= s[0] <= 1 and 0 <= s[1] <= 1]
print(sol)

[(sqrt(5)/5, 2*sqrt(5)/5)]


Example polynomials: </p>
$ f(x,y) = 2 + 3x - x^2 + 5x^3 - 15y + 2y^2 + 8y^3 + 20xy + x^2y + 12xy^2 + 100x^2y^2 + 1000x^3y^2 + 598x^2y^3 - 2105x^3y^3$ </br>
$ g(x,y) = x + x^2 + 2x^3 + y - 5y^2 -27.7xy^2 - 3x^2y^2 $ </p> 

Solution: $[x, y] = [0.1, 0.2]$ 

In [57]:
np.set_printoptions(precision=12)
x,y = sp.symbols('x, y')
eqs = [2 + 3*x - x**2 + 5*x**3 - 15*y + 2*y**2 + 8*y**3 + 20*x*y + x**2*y + 12*x*y**2 + 100*x**2*y**2 + 1000*x**3*y**2 + 598*x**2*y**3 - 2105*x**3*y**3, x + x**2 + 2*x**3 + y - 5*y**2 -27.7*x*y**2 - 3*x**2*y**2 ]
sol = sp.solve(eqs, (x, y))

#Show only the with x and y between 0 and 1
sol = [s for s in sol if 0 <= s[0] <= 1 and 0 <= s[1] <= 1]
print(sol)

[(0.100000000000000, 0.200000000000000)]


Example polynomials: </p>
$ f(x,y,z) = -1.2568 + x + x^2 + 5x^3 - y + 3y^2 + 4z + xy + xyz^2$ </br>
$ g(x,y,z) = -0.3129 + x^3 + 8y^2 - z^3 + yz^2 + 3x^3z $ </br> 
$ h(x,y,z) = 0.983 + 5x^3 - y + 8y^2 - 4z + z^2 + x^2y $ </p>

Solution: $[x, y, z] = [0.1, 0.2, 0.3]$ 

In [46]:
x,y,z= sp.symbols('x, y, z')
eqs = [-1.2568 + x + x**2 + 5*x**3 - y + 3*y**2 + 4*z + x*y + x*y*z**2, -0.3129 + x**3 + 8*y**2 - z**3 + y*z**2 + 3*x**3*z, 0.983 + 5*x**3 - y + 8*y**2 - 4*z + z**2 + x**2*y]
sol = sp.solve(eqs, (x, y, z))

#Show only the with x and y between 0 and 1
sol = [s for s in sol if 0 <= s[0] <= 1 and 0 <= s[1] <= 1]
print(sol)

[(0.100000000000000, 0.200000000000000, 0.300000000000000)]


In [56]:
np.set_printoptions(precision=12)
x = 0.2000000000000
y = 0.3000000000000
f = -2 + x + x**2 + 7*x**3 + 2*y + x*y + 8*x**2*y - 2*x**3*y - 5*y**2 + x*y**2 - x**2*y**2 + 3*x**3*y**2 - y**3 + x*y**3 + x**2*y**3 + 12*x**3*y**3
print(f)

# detivative of f with respect to x
f_x = 1 + 2*x + 21*x**2 + y + 16*x*y - 6*x**2*y + y**2 - 2*x*y**2 + 9*x**2*y**2 + y**3 + 2*x*y**3 + 36*x**2*y**3
print(f_x)
# detivative of f with respect to y
f_y = 2 + x + 8*x**2 - 2*x**3 - 10*y + 2*x*y - 2*x**2*y + 6*x**3*y - 3*y**2 + 3*x*y**2 + 3*x**2*y**2 + 36*x**3*y**2
print(f_y)

-1.404168
3.59108
-0.5648799999999994
